In [1]:
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('movies_metabase_clean.csv')

In [3]:
data.columns

Index(['id', 'title', 'original_title', 'status', 'release_date', 'company',
       'revenue', 'budget', 'adult', 'belongs_to_collection',
       'original_language', 'popularity', 'country', 'runtime',
       'spoken_languages', 'tagline', 'video', 'vote_average', 'vote_count'],
      dtype='object')

In [4]:
columns=['id', 'title', 'original_title', 'status', 'release_date', 'company',
       'revenue', 'budget', 'adult', 'belongs_to_collection',
       'original_language', 'popularity', 'country', 'runtime',
       'spoken_languages', 'tagline', 'video', 'vote_average', 'vote_count']

In [5]:
target=["revenue","release_date"]

In [6]:
data.head()

,id,title,original_title,status,release_date,company,revenue,budget,adult,belongs_to_collection,original_language,popularity,country,runtime,spoken_languages,tagline,video,vote_average,vote_count
0,862,Toy Story,Toy Story,Released,1995-10-30,Pixar Animation Studios,373554033.0,30000000,False,True,en,21.946943,US,81.0,1,NaN,False,7.7,5415.0
1,8844,Jumanji,Jumanji,Released,1995-12-15,TriStar Pictures,262797249.0,65000000,False,False,en,17.015539,US,104.0,2,Roll the dice and unleash the excitement!,False,6.9,2413.0
2,31357,Waiting to Exhale,Waiting to Exhale,Released,1995-12-22,Interscope Communications,81452156.0,16000000,False,False,en,3.859495,US,127.0,1,Friends are the people who let you be yourself...,False,6.1,34.0
3,11862,Father of the Bride Part II,Father of the Bride Part II,Released,1995-02-10,Twentieth Century Fox Film Corporation,76578911.0,0,False,True,en,8.387519,US,106.0,1,Just When His World Is Back To Normal... He's ...,False,5.7,173.0
4,949,Heat,Heat,Released,1995-12-15,Sandollar Productions,187436818.0,60000000,False,False,en,17.924927,US,170.0,2,A Los Angeles Crime Saga,False,7.7,1886.0


In [10]:
# Create our features

X = data.drop(columns='revenue')
x= data.drop(columns='release_date')

# Create our target
y = data.loc[:, target].copy()

In [11]:
X.describe()

,id,budget,popularity,runtime,spoken_languages,vote_average,vote_count
count,7397.000000,7.397000e+03,7397.000000,7391.000000,7397.000000,7397.000000,7397.000000
mean,66978.832770,2.260451e+07,8.516839,107.719659,1.446803,6.218224,559.914830
std,98989.419782,3.695024e+07,12.166210,21.480504,0.893634,1.025649,1106.682445
min,5.000000,0.000000e+00,0.000001,0.000000,0.000000,0.000000,0.000000
25%,9602.000000,0.000000e+00,3.933132,94.000000,1.000000,5.700000,41.000000
50%,19158.000000,7.500000e+06,7.436001,104.000000,1.000000,6.300000,161.000000
75%,71825.000000,2.800000e+07,10.921430,118.000000,2.000000,6.900000,542.000000
max,443319.000000,3.800000e+08,547.488298,338.000000,9.000000,10.000000,14075.000000


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

NameError: name 'train_test_split' is not defined

In [13]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

NameError: name 'X_train' is not defined

In [ ]:
# Splitting into Train and Test sets into an 80/20 split.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)

In [ ]:
# Determine the shape of our training and testing sets.
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

In [ ]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

In [ ]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))